## Data Transformation

In [1]:
import os
%pwd
os.chdir("../")
%pwd


'd:\\Data Science\\END to END Proj\\QA-BOT'

In [2]:
from dataclasses import dataclass
from pathlib import Path
from typing import List, Dict, Tuple
import json
import os
from src.QABOT import logger
import numpy as np
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import DistilBertTokenizerFast
import torch
from torch.utils.data import Dataset
from tqdm import tqdm

d:\Data Science\END to END Proj\QA-BOT\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    TRAIN_FILE: Path
    DEV_FILE: Path
    MAX_LEN: int
    DOC_STRIDE: int
    PARA_VECTORIZER_FILE: Path
    SENT_VECTORIZER_FILE: Path
    PARAGRAPHS_FILE: Path
    SENTENCES_FILE: Path

In [4]:
from src.QABOT.constant import *
from src.QABOT.utils.common import read_yaml,create_directories 
import json
from src.QABOT import logger
import joblib


In [5]:



class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
     ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        

        create_directories([self.config.artifacts_root])


    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=Path(config.root_dir),
            TRAIN_FILE=Path(config.TRAIN_FILE),
            DEV_FILE=Path(config.DEV_FILE),
            MAX_LEN=self.params.Transformation.MAX_LEN,
            DOC_STRIDE=self.params.Transformation.DOC_STRIDE,
            PARA_VECTORIZER_FILE=Path(config.PARA_VECTORIZER_FILE),
            SENT_VECTORIZER_FILE=Path(config.SENT_VECTORIZER_FILE),
            PARAGRAPHS_FILE=Path(config.PARAGRAPHS_FILE),
            SENTENCES_FILE=Path(config.SENTENCES_FILE)
        )
        return data_transformation_config

In [6]:
import nltk


class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        try:
            nltk.data.find('tokenizers/punkt')
            nltk.data.find('tokenizers/punkt_tab')  # Specifically check for punkt_tab
        except LookupError:
            nltk.download('punkt', quiet=True)
            nltk.download('punkt_tab', quiet=True)  # Download punkt_tab specifically
          
        self.tokenizer = DistilBertTokenizerFast.from_pretrained(
            "distilbert-base-uncased",
            use_auth_token=""  # Replace with your actual token
        )

    def flatten_squad(self, squad_js: Dict) -> List[Dict]:
        """Convert SQuAD format to flattened QA pairs"""
        rows = []
        for article in squad_js["data"]:
            for para in article["paragraphs"]:
                context = para["context"]
                for qa in para["qas"]:
                    if "answers" not in qa or len(qa["answers"]) == 0:
                        continue
                    ans = qa["answers"][0]
                    rows.append({
                        "id": qa["id"],
                        "question": qa["question"],
                        "context": context,
                        "answer_text": ans["text"],
                        "answer_start": ans["answer_start"],
                    })
        return rows

    def collect_paragraphs(self, squad_list: List[Dict]) -> List[str]:
        """Extract all unique paragraphs from SQuAD data"""
        paras = []
        for article in squad_list:
            for para in article["paragraphs"]:
                paras.append(para["context"])
        return list(dict.fromkeys(paras))  # Deduplicate

    def build_tfidf_retriever(self, paragraphs: List[str]) -> Tuple:
        """Create TF-IDF vectorizer and vectorize paragraphs"""
        vectorizer = TfidfVectorizer(lowercase=True, stop_words="english")
        tfidf_matrix = vectorizer.fit_transform(paragraphs)
        return vectorizer, tfidf_matrix

    def build_sentence_bank(self, paragraphs: List[str]) -> Tuple:
        """Create sentence-level TF-IDF index"""
        all_sentences = []
        sent2para_idx = []
        for p_i, p in enumerate(paragraphs):
            sents = sent_tokenize(p)
            all_sentences.extend(sents)
            sent2para_idx.extend([p_i]*len(sents))
        
        vectorizer = TfidfVectorizer(lowercase=True, stop_words="english")
        tfidf_matrix = vectorizer.fit_transform(all_sentences)
        return vectorizer, tfidf_matrix, all_sentences

    def prepare_features(self, rows: List[Dict]) -> List[Dict]:
        """Convert QA pairs to model input features with overflow handling"""
        features = []
        for r in tqdm(rows, desc="Tokenizing examples"):
            question = r["question"]
            context = r["context"]
            answer_text = r["answer_text"]
            answer_start = r["answer_start"]
            answer_end = answer_start + len(answer_text)

            tokenized = self.tokenizer(
                question,
                context,
                truncation="only_second",
                max_length=self.config.MAX_LEN,
                stride=self.config.DOC_STRIDE,
                return_overflowing_tokens=True,
                return_offsets_mapping=True,
                padding="max_length",
            )

            for i in range(len(tokenized["input_ids"])):
                offsets = tokenized["offset_mapping"][i]
                input_ids = tokenized["input_ids"][i]
                attention_mask = tokenized["attention_mask"][i]
                seq_ids = tokenized.sequence_ids(i)

                # Find context span
                context_start = next((idx for idx, s in enumerate(seq_ids) if s == 1), None)
                context_end = next((idx for idx in reversed(range(len(seq_ids))) if seq_ids[idx] == 1), None)

                if context_start is None or context_end is None:
                    continue

                # Locate answer span
                cls_index = input_ids.index(self.tokenizer.cls_token_id)
                start_token = end_token = cls_index

                for idx in range(context_start, context_end + 1):
                    start_char, end_char = offsets[idx]
                    if start_char <= answer_start and end_char >= answer_start:
                        start_token = idx
                    if start_char <= answer_end-1 and end_char >= answer_end-1:
                        end_token = idx

                if start_token != cls_index or end_token != cls_index:
                    features.append({
                        "input_ids": torch.tensor(input_ids),
                        "attention_mask": torch.tensor(attention_mask),
                        "start_positions": torch.tensor(start_token),
                        "end_positions": torch.tensor(end_token),
                    })
        return features

    def transform(self):
        """Execute full transformation pipeline"""
        try:
            # Load raw data
            with open(self.config.TRAIN_FILE, "r", encoding="utf-8") as f:
                train_squad = json.load(f)
            with open(self.config.DEV_FILE, "r", encoding="utf-8") as f:
                dev_squad = json.load(f)

            # Flatten QA pairs
            train_rows = self.flatten_squad(train_squad)
            dev_rows = self.flatten_squad(dev_squad)
            logger.info(f"Flattened {len(train_rows)} train and {len(dev_rows)} dev examples")

            # Build retrieval corpus
            all_paragraphs = self.collect_paragraphs(train_squad["data"]) + self.collect_paragraphs(dev_squad["data"])
            para_vectorizer, para_tfidf = self.build_tfidf_retriever(all_paragraphs)
            sent_vectorizer, sent_tfidf, all_sentences = self.build_sentence_bank(all_paragraphs)
            logger.info(f"Built TF-IDF retriever with {len(all_paragraphs)} paragraphs and {len(all_sentences)} sentences")

            # Prepare model features
            train_features = self.prepare_features(train_rows)
            dev_features = self.prepare_features(dev_rows)
            logger.info(f"Prepared {len(train_features)} train and {len(dev_features)} dev features")

            # Save artifacts
            os.makedirs(self.config.root_dir, exist_ok=True)
            joblib.dump(para_vectorizer, self.config.PARA_VECTORIZER_FILE)
            joblib.dump(sent_vectorizer, self.config.SENT_VECTORIZER_FILE)
            with open(self.config.PARAGRAPHS_FILE, "w", encoding="utf-8") as f:
                json.dump(all_paragraphs, f, ensure_ascii=False)
            with open(self.config.SENTENCES_FILE, "w", encoding="utf-8") as f:
                json.dump(all_sentences, f, ensure_ascii=False)
            
            return QADataset(train_features), QADataset(dev_features)

        except Exception as e:
            logger.error(f"Data transformation failed: {str(e)}")
            raise e

class QADataset(Dataset):
    """PyTorch Dataset for QA features"""
    def __init__(self, features):
        self.features = features
    def __len__(self):
        return len(self.features)
    def __getitem__(self, idx):
        return self.features[idx]


In [8]:

if __name__ == "__main__":
    try:
        config_manager = ConfigurationManager()
        transformation_config = config_manager.get_data_transformation_config()
        transformer = DataTransformation(config=transformation_config)
        train_dataset, dev_dataset = transformer.transform()
        logger.info(" Data transformation completed successfully")
    except Exception as e:
        logger.error(f"Data transformation failed: {str(e)}")
        raise e

[2025-08-13 16:18:41,593: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-13 16:18:41,593: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-13 16:18:41,593: INFO: common: created directory at: artifacts]
[2025-08-13 16:18:41,593: INFO: common: created directory at: artifacts/data_transformation]


d:\Data Science\END to END Proj\QA-BOT\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1908: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


[2025-08-13 16:18:43,391: INFO: 232401020: Flattened 87599 train and 10570 dev examples]
[2025-08-13 16:18:51,273: INFO: 232401020: Built TF-IDF retriever with 20958 paragraphs and 104034 sentences]


Tokenizing examples: 100%|██████████| 10570/10570 [00:15<00:00, 664.84it/s]

[2025-08-13 16:21:10,443: INFO: 232401020: Prepared 87790 train and 10616 dev features]


[2025-08-13 16:21:12,825: INFO: 4003408384:  Data transformation completed successfully]
